In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../Includes/configuration"

In [0]:
%run "../Includes/common_functions"

In [0]:
race_results_list = spark.read\
    .format("delta").load((f"{presentation_folder_path}/race_results"))\
    .withColumnRenamed("result_file_date", "file_date")\
    .filter(f"file_date = '{v_file_date}'")\
    .select("race_year")\
    .distinct()\
    .collect()


In [0]:
race_year_list = []

for race_year in race_results_list:
    race_year_list.append(race_year.race_year)


In [0]:
df = spark.read\
    .format("delta").load((f"{presentation_folder_path}/race_results"))\
    .filter(col("race_year").isin(race_year_list))

In [0]:
driver_standings_df = df\
    .groupBy("race_year", "driver_name", "driver_nationality")\
        .agg(sum("points").alias("total_points"),
                 count(when(col("position") == 1, True)).alias("wins"))

In [0]:
window_spec = Window.partitionBy("race_year").orderBy(col("total_points").desc(), col("wins").desc())

In [0]:
df_final = driver_standings_df.withColumn("rank", rank().over(window_spec))


In [0]:
#overwrite_partition(df_final, "f1_presentation", "driver_standings", "race_year")

In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year"
merge_delta_data(df_final, "f1_presentation", "driver_standings", presentation_folder_path, merge_condition, "race_year")

In [0]:
%sql
select * from f1_presentation.driver_standings
where race_year = 2020
order by wins desc

race_year,driver_name,driver_nationality,total_points,wins,rank
2020,Lewis Hamilton,British,347.0,11,1
2020,Valtteri Bottas,Finnish,223.0,2,2
2020,Max Verstappen,Dutch,214.0,2,3
2020,Pierre Gasly,French,75.0,1,10
2020,Sergio Pérez,Mexican,125.0,1,4
2020,Daniel Ricciardo,Australian,119.0,0,5
2020,Nicholas Latifi,Canadian,0.0,0,21
2020,Esteban Ocon,French,62.0,0,12
2020,Charles Leclerc,Monegasque,98.0,0,8
2020,Nico Hülkenberg,German,10.0,0,15
